In [7]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [8]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [10]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)

# Read the CSV into a DataFrame
home_sales_df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# 2. Create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView("home_sales")

# To verify if it worked, you can run a simple query like:
spark.sql("SELECT * FROM home_sales LIMIT 5").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [11]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql import functions as F

# 1. Filter for houses with four bedrooms
four_bedroom_houses_df = home_sales_df.filter(home_sales_df['bedrooms'] == 4)

# 2. Extract year from the 'date' column (assuming 'date' is in the format 'yyyy-MM-dd')
four_bedroom_houses_df = four_bedroom_houses_df.withColumn('year', F.year(four_bedroom_houses_df['date']))

# 3. Group by the 'year' and calculate the average price
average_price_per_year_df = four_bedroom_houses_df.groupBy('year').agg(
    F.round(F.avg('price'), 2).alias('average_price')
)

# 4. Show the result
average_price_per_year_df.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
+----+-------------+



In [13]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
from pyspark.sql import functions as F

# 1. Filter for homes with 3 bedrooms and 3 bathrooms
three_bed_three_bath_df = home_sales_df.filter((home_sales_df['bedrooms'] == 3) &
                                               (home_sales_df['bathrooms'] == 3))

# 2. Group by the 'year_built' column and calculate the average price
average_price_per_year_built_df = three_bed_three_bath_df.groupBy('date_built').agg(
    F.round(F.avg('price'), 2).alias('average_price')
)

# 3. Show the result
average_price_per_year_built_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [14]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
from pyspark.sql import functions as F

# 1. Filter for homes with 3 bedrooms, 3 bathrooms, 2 floors, and greater than or equal to 2,000 square feet
filtered_homes_df = home_sales_df.filter(
    (home_sales_df['bedrooms'] == 3) &
    (home_sales_df['bathrooms'] == 3) &
    (home_sales_df['floors'] == 2) &
    (home_sales_df['sqft_living'] >= 2000)
)

# 2. Group by the 'year_built' column and calculate the average price
average_price_per_year_built_df = filtered_homes_df.groupBy('date_built').agg(
    F.round(F.avg('price'), 2).alias('average_price')
)

# 3. Show the result
average_price_per_year_built_df.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
import time
from pyspark.sql import functions as F

# Start the timer to measure the query execution time
start_time = time.time()

# 1. Group by the 'view' column and calculate the average price
average_price_by_view_df = home_sales_df.groupBy('view').agg(
    F.round(F.avg('price'), 2).alias('average_price')
)

# 2. Filter the data to include only average prices greater than or equal to $350,000
filtered_df = average_price_by_view_df.filter(average_price_by_view_df['average_price'] >= 350000)

# 3. Order the results by descending 'view' rating
ordered_df = filtered_df.orderBy('view', ascending=False)

# 4. Show the result
ordered_df.show()

# End the timer and print the execution time
end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 2.0745136737823486 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
# 1. Cache the DataFrame
home_sales_df.cache()

# 2. Create or refresh the temporary view for the DataFrame
home_sales_df.createOrReplaceTempView("home_sales")

# Now the table is cached and can be used for further queries


In [17]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [18]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
import time

# 1. Start the timer
start_time = time.time()

# 2. Query to calculate average price of a home per "view" rating
query = """
SELECT view, ROUND(AVG(price), 2) AS avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

# 3. Execute the query
result = spark.sql(query)

# 4. Show the result (if desired, you can adjust this for better visualization)
result.show()

# 5. Print the runtime
print("--- %s seconds ---" % (time.time() - start_time))



+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 2.6174376010894775 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# 1. Ensure that the 'date_built' field is in the correct format if it's not already.
# If the date is in string format, you may need to convert it to a timestamp.

# Convert 'date_built' to DateType if it's not already in that format (adjust if needed)
from pyspark.sql.functions import col, to_date

home_sales_df = home_sales_df.withColumn("date_built", to_date(col("date_built"), "yyyy-MM-dd"))

# 2. Write the DataFrame to Parquet, partitioned by the 'date_built' field.
home_sales_df.write.partitionBy("date_built").parquet("path/to/output/parquet_home_sales")

# Ensure that the directory exists or specify the path where the Parquet files should be stored.


In [20]:
# 11. Read the parquet formatted data.
# Read the parquet formatted data into a DataFrame
parquet_home_sales_df = spark.read.parquet("path/to/output/parquet_home_sales")

# Show the first few rows of the DataFrame to confirm the data
parquet_home_sales_df.show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|f8a53099-ba1c-47d...|2022-04-08|936923|       4|        3|       3167|   11733|     2|         1|  76|      NULL|
|7530a2d8-1ae3-451...|2021-06-13|379628|       2|        2|       2235|   14384|     1|         0|  23|      NULL|
|43de979c-0bf0-4c9...|2019-04-12|417866|       2|        2|       2127|   10575|     2|         0|   0|      NULL|
|b672c137-b88c-48b...|2019-10-16|239895|       2|        2|       1631|   11149|     2|         0|   0|      NULL|
|e0726d4d-d595-407...|2022-01-08|424418|       3|        2|       2249|   13878|     2|         0|   4|      NULL|
|5aa00529-0533-46b...|2019-01-30|218712|       2|        3|       1965|   14375|

In [21]:
# 12. Create a temporary table for the parquet data.
# Create a temporary view (table) for the parquet data
parquet_home_sales_df.createOrReplaceTempView("home_sales_parquet")

# You can now run SQL queries on this temporary table
spark.sql("SELECT * FROM home_sales_parquet LIMIT 10").show()


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|f8a53099-ba1c-47d...|2022-04-08|936923|       4|        3|       3167|   11733|     2|         1|  76|      NULL|
|7530a2d8-1ae3-451...|2021-06-13|379628|       2|        2|       2235|   14384|     1|         0|  23|      NULL|
|43de979c-0bf0-4c9...|2019-04-12|417866|       2|        2|       2127|   10575|     2|         0|   0|      NULL|
|b672c137-b88c-48b...|2019-10-16|239895|       2|        2|       1631|   11149|     2|         0|   0|      NULL|
|e0726d4d-d595-407...|2022-01-08|424418|       3|        2|       2249|   13878|     2|         0|   4|      NULL|
|5aa00529-0533-46b...|2019-01-30|218712|       2|        3|       1965|   14375|

In [22]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
import time

# Start the timer for runtime calculation
start_time = time.time()

# Running the query on the parquet data
result_df = spark.sql("""
    SELECT view,
           ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

# Show the results
result_df.show()

# Print the runtime for the query execution
print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.8046927452087402 seconds ---


In [23]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [24]:
# 15. Check if the home_sales is no longer cached
# Check if the 'home_sales' table is cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("The 'home_sales' table is still cached.")
else:
    print("The 'home_sales' table is no longer cached.")


The 'home_sales' table is no longer cached.
